Creates the famous Mandelbrot set using linear interpolation for colour smoothing. From Blue->Purple->Red->Orange->Yellow

Imports


In [13]:
from PIL import Image, ImageDraw
import math
import numpy as np

Functions

In [14]:
def createIter(currentPixelX, currentPixelY, maxiter):
    # Creates the next value to be used in the loop
    nextvalue = complex(0, 0)
    # Loops through the max iterations checking if the point converges 
    for j in range(maxiter):
        # Turns the current pixel we are on and make it a complex number 
        startingVal = complex(currentPixelX, currentPixelY)
        # The mandelbrot set equation
        nextvalue = nextvalue ** 2 + startingVal
        # Checks if it has converged and returns the value it diverged on (will be coloured)
        if nextvalue.real > 2 or nextvalue.real < -2:
            return j, nextvalue
    # If we reached max_iter that means it didnt diverge so will be black
    return maxiter, nextvalue


# Computes a continuous colour value for a point based on the iterations it took if iter==max_iter then return 0
def ratio(c, iterations, max_iterations):
    if iterations < max_iterations:
        return iterations + 1 - math.log(math.log(abs(c))) / math.log(2)
    return 0

# Generates the graident for colouring
def generate_gradient(colors, steps):
    # Create an empty aray
    gradient = []
    n = len(colors) - 1
    # How many colours we want to generate
    for i in range(steps):
        # Determine the segment and relative position within the segment
        segment = i * n / steps
        # Ensures that the index does not exceed the size of the list
        segment_index = min(int(segment), n - 1)
        t = segment - segment_index

        # Interpolate the colour of succeeding colours so if it was blue&purple generate a colour somewhere in the range of that based on i
        start_color = colors[segment_index]
        end_color = colors[segment_index + 1]
        # Now after selecting the colours (say blue and purple) we bring i into this and come up with a linear interpolated value
        r = int(start_color[0] + (end_color[0] - start_color[0]) * t)
        g = int(start_color[1] + (end_color[1] - start_color[1]) * t)
        b = int(start_color[2] + (end_color[2] - start_color[2]) * t)
        # Add the tuple to the array
        gradient.append((r, g, b))
    
    return gradient


def smoothInterpolateColor(value, ratio):
    # Crerates the colour array that basically goes from blue->purple->red->orange->yellow
    # generates all 50 of the colours that make it a smoother transition from the above flow
    colours = np.array(generate_gradient(
        [(0, 0, 128), 
         (128, 0, 128), 
         (255, 0, 0), 
         (255, 165, 0), 
         (255, 255, 0)], 
         100))
    # Selects the first colour based on the "value"
    index1 = min(len(colours) - 1, int(value))
    # Selects the next value based on "value"
    index2 = min(len(colours) - 1, int(value + 1))
    # now interpolate every aspect of rgb from the first index to the second index to generate a smoother transition
    red = int(colours[index2, 0] * ratio + colours[index1, 0] * (1 - ratio))
    green = int(colours[index2, 1] * ratio + colours[index1, 1] * (1 - ratio))
    blue = int(colours[index2, 2] * ratio + colours[index1, 2] * (1 - ratio))
    return red, green, blue

def createpicture(w,h, xneg, yneg, xpos, ypos, maxiter, img):
    # Loop through the width of the picture
    for i in range(w):
        # Gets the current real value we are on
        currentPixelX = xneg + i / w * 3
        # Loops through the height of the picture 
        for x in range(h):
            # gets the current imaginary value we are on
            currentPixelY = yneg + x / h * 3
            # Returns the iteration it returned on and the value at the end
            col, complex_val = createIter(currentPixelX, currentPixelY, maxiter)
            # Turns the complex val into a value to be used for colouring
            rat = ratio(complex_val, col, maxiter)
            # maps the smooth ratio to a color from a gradient, ensuring smooth transitions between colors as the smooth ratio change
            colour = smoothInterpolateColor(rat, rat % 1.0)
            # if the maxiter was reached then it didnt diverge thus black
            if (col == maxiter):
                img.point((i,x), (0,0,0))
            else: img.point((i, x), colour) # else colour it based on the colour returned
    return img

Now the code for the mandelbrot set

In [15]:
# Defines the bordes of the images and the iterations
xneg = -2
xpos = 1
yneg = -1.5
ypos = 1.5
maxiter = 1000
h = 1660
w = 1660

# Creates the image pbject
img = Image.new("RGB", (h, w), "white")
draw = ImageDraw.Draw(img)
# Creates the mandelbrot set picture
createpicture(w,h, xneg, yneg, xpos, ypos, maxiter, draw)

# Resizes the information
img = img.resize((800, 800), Image.LANCZOS)
# Saves the image
img.save("mandelbrot.png", "png")
print("Image Saved")

Image Saved


Future Considerations:
Using numpy special functionality can severely reduce the run time of this program and make it more efficient